In [1078]:
import numpy as np

In [1079]:
import sys
sys.path.append(r"C:\_Lib\python")

from mask_maker import * 
# from slab.circuits.mp_components import *

In [1080]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1081]:
from mm_qubit import *
from mm_chips_base import * 

# Imports

# Mask Generation

## Wafer Initialization

 4in wafer -> diameter=101600, flat_distance=48200


 3in wafer -> diameter=76200., flat_distance=37100.

 
 2in wafer -> diameter=50800., flat_distance=24100.

In [1082]:
MaskName = 'TestingQubit1113_junc_'
save_folder=r"C:\Users\eeshg\Documents\multimode_sims\Design\coupler\designs_dxf"
m = WaferMask(MaskName, diameter=50800., flat_angle=270., flat_distance=24100., wafer_padding=2500,
                  chip_size=(1600, 25950),
                  dicing_border=400, etchtype=False, wafer_edge=False,
                  dashed_dicing_border=80, ndashes=3, dice_corner=True, square_arr=False)

Maximum number of rows=1 and cols=22
Room for 18 chips on wafer.


## Chips

In [1083]:
## Iniatilization
mm_chips_base = MMChipsBase()
chip_defaults = mm_chips_base.set_mask_init()

### Chip Configs

In [1084]:
chip_configs = [
        # (junc_correction, flip, CHIPNAME, type) # type does not matter 
        ({'width': 0.2, 'gap': 0.1}, False, 'C1A', 1),
        ({'width': 0.3, 'gap': 0.1}, False, 'C1B', 1),
        ({'width': 0.4, 'gap': 0.1}, False, 'C1C', 1),
        ({'width': 0.25, 'gap': 0.15}, False, 'C1D', 1),
        ({'width': 0.45, 'gap': 0.15}, False, 'C1E', 1),
        ({'width': 0.35, 'gap': 0.15}, False, 'C1F', 1),
        ({'width': 0.5, 'gap': 0.2}, False, 'C1G', 0),
        ({'width': 0.45, 'gap': 0.2}, False, 'C1H', 0),
        ({'width': 0.4, 'gap': 0.2}, False, 'C1I', 0),
        ({'width': 0.35, 'gap': 0.2}, False, 'C1J', 1),
        # ({'width': 0.14, 'gap': 0.2}, False, 'C1K', 1),
        # ({'width': 0.35, 'gap': 0.2}, False, 'C1L', 1),
        # ({'width': 0.25, 'gap': 0.2}, False, 'C1M', 1),
        # ({'width': 0.2, 'gap': 0.4}, False, 'C1N', 0),
    ][:10]

## Configs specific to Multimode Qubit Chip

In [1085]:
qubit_optical_configs= {
    'x_thin': 30,
    'y_right_thin': 350,
    'x_right_thick': 150,
    'y_right_thick': 3100,
    'y_left_thin': 370,
    'x_left_thick': 150,
    'y_left_thick': 3700,
    'jjgap': 30,
    'xpos': 1000 - 200,
    'ypos': 17700 - 200 + 7780,
    
}

In [1086]:
test_qubit_optical_configs = {
    'x_thin': 30,
    'y_right_thin': 100,
    'x_right_thick': 800,
    'y_right_thick': 800,
    'y_left_thin': 100,
    'x_left_thick': 800,
    'y_left_thick': 800,
    'jjgap': 30,
}
test_qubit_optical_posns = [
    (800, 3000+200), 
    (800, 5500+200),
    (800, 8000+200),
]

In [1087]:

qubit_junction_configs = dict(
    bar_width=40,
    bar_length=15,
    thin_bar_length = 2,
    taper_length=10,
    jj_length=0.8,
    # jj_width=junc_correction['width'],
    # jj_gap=junc_correction['gap'],
    top_bar_jj_width=5,
    # under cut parameters 
    undercut_width = 0.2,
    # for horizontal oriented junctions 07/27 blunder
    horizontal_l_arm = 70+15,
    bend_radius = 5,

)


### Chip Generation

In [1088]:
draw_optical = False
# draw_optical = False
draw_ebeam = True
# draw_ebeam = False
two_layer = True if draw_ebeam == 1 else False

In [1089]:
chips = []
author = 'Eesh Gupta'
# flag flips the junction vertically 
for idx, (junc_correction, flip, name, type) in enumerate(chip_configs):
    chip = Chip(
        name,
        author="",
        size=m.chip_size,
        mask_id_loc=(5800, 6430),
        chip_id_loc=(200, 100),
        textsize=(70, 70),
        two_layer=two_layer,
        solid=False,
    )
    full_chip_drawer = MultimodeQubitChip(structure=chip, chip_defaults=chip_defaults, flip=flip)
    full_chip_drawer.draw_alignment_marks()

    if draw_optical:
        full_chip_drawer.draw_multimode_qubit_optical(qubit_optical_configs=qubit_optical_configs,
                                                      flag = not flip,xpos= qubit_optical_configs['xpos'],
                                                      ypos= qubit_optical_configs['ypos'])
        #test structures
        for idx, (xpos, ypos) in enumerate(test_qubit_optical_posns):
            full_chip_drawer.draw_multimode_qubit_optical(qubit_optical_configs=test_qubit_optical_configs,
                                                          flag = not flip, 
                                                          xpos = xpos, ypos = ypos)
        
    if draw_ebeam:
        # print(f"Drawing e-beam features for {name} with flip={flip}")
        full_chip_drawer.draw_multimode_qubit_ebeam(junction_configs=qubit_junction_configs,
                                                    junc_correction=junc_correction, flag= not flip, 
                                                    xpos = qubit_optical_configs['xpos'],
                                                    ypos = qubit_optical_configs['ypos'] - qubit_optical_configs['y_left_thick'] - qubit_optical_configs['y_left_thin'] - qubit_optical_configs['jjgap'] )#- qubit_optical_configs['x_thin'])
        ##test structures
        for idx, (xpos, ypos) in enumerate(test_qubit_optical_posns):
            full_chip_drawer.draw_multimode_qubit_ebeam(junction_configs=qubit_junction_configs,
                                                        junc_correction=junc_correction, flag= not flip,
                                                        xpos = xpos, 
                                                        ypos = ypos - test_qubit_optical_configs['y_left_thick'] - test_qubit_optical_configs['y_left_thin'] - test_qubit_optical_configs['jjgap'] )#- test_qubit_optical_configs['x_thin'])

    # draw_bridge_junction(chip, d, junc_correction, flip)
    chips.append(chip)





flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0
flag1 0


In [1090]:
square

0

## Addings Chips to wafer

In [1091]:
# add chips to wafer
chip_fnames = []
for idx, c in enumerate(chips[:18]):
    name = m.add_chip(c, 1, label= True, save_folder=save_folder)
    chip_fnames.append(name)
chip_fnames

[<mask_maker.sdxf.Layer object at 0x00000277E9841E50>, <mask_maker.sdxf.Layer object at 0x00000277FB0058D0>, <mask_maker.sdxf.Layer object at 0x00000277FB361E10>]
GAP
PIN
[<mask_maker.sdxf.Layer object at 0x00000277E9841E50>, <mask_maker.sdxf.Layer object at 0x00000277FB3FE1D0>, <mask_maker.sdxf.Layer object at 0x00000277FB3FE250>]
GAP
PIN
[<mask_maker.sdxf.Layer object at 0x00000277E9841E50>, <mask_maker.sdxf.Layer object at 0x00000277FB3D2850>, <mask_maker.sdxf.Layer object at 0x00000277FB3D28D0>]
GAP
PIN
[<mask_maker.sdxf.Layer object at 0x00000277E9841E50>, <mask_maker.sdxf.Layer object at 0x00000277FB396C90>, <mask_maker.sdxf.Layer object at 0x00000277FB396D10>]
GAP
PIN
[<mask_maker.sdxf.Layer object at 0x00000277E9841E50>, <mask_maker.sdxf.Layer object at 0x00000277FB46D9D0>, <mask_maker.sdxf.Layer object at 0x00000277FB46F390>]
GAP
PIN
[<mask_maker.sdxf.Layer object at 0x00000277E9841E50>, <mask_maker.sdxf.Layer object at 0x00000277FBBDF7D0>, <mask_maker.sdxf.Layer object at 0x0

['C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1A.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1B.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1C.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1D.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1E.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1F.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1G.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1H.dxf',
 'C:\\Users\\eeshg\\Documents\\multimode_sims\\Design\\coupler\\designs_dxf\\TESTINGQUBIT1113_JUNC_-C1I.dxf',
 'C:\\User

# Output Wafer

In [1092]:
import os
import subprocess
from time import sleep

def save_and_open_mask(m, folder_path, klayout_path, sleep_time=0.1):

    mask_name = m.name
    mask_file_path = os.path.join(folder_path, mask_name + '.dxf')
    m.save(name=mask_file_path)

    print("\n\n Chip names are:")
    print("_____________________")
    # for name in chip_names:
    #     print(name)
    print("_____________________\n\n")

    sleep(sleep_time)

    def open_klayout_app(klayout_path, mask_file_path):
        subprocess.Popen(f'"{klayout_path}" "{mask_file_path}" -e')

    open_klayout_app(klayout_path, mask_file_path)

# Usage:
folder_path = r"C:\Users\eeshg\Documents\multimode_sims\Design\coupler\designs_dxf"
klayout_path = r"C:\Users\eeshg\AppData\Roaming\KLayout\klayout_app"
save_and_open_mask(m, folder_path, klayout_path)




 Chip names are:
_____________________
_____________________




In [1093]:
2/2+1

2.0